# Boosting – Grundidee

Boosting erzeugt ein starkes Modell, indem es **viele schwache Modelle** (meist flache Bäume)
**nacheinander** trainiert.

Das Wichtige:  
Jeder neue Baum **korrigiert gezielt die Fehler des bisherigen Modells**.


## Mathematische Sicht

Wir konstruieren ein Modell als additive Funktion:

$$
F_T(x) = \sum_{t=1}^T \alpha_t \, h_t(x)
$$

wobei  
- $ h_t(x) $ = schwacher Lerner (z. B. Decision Stump)  
- $ \alpha_t $ = Gewicht des t-ten Baums  
- $ T $ = Anzahl der Boosting-Schritte  

Alle drei Boosting-Methoden unterscheiden sich nur darin:  
 **Wie werden $ h_t(x) $ ausgewählt?**  
 **Wie werden Fehler gewichtet?**  
 **Wie wird $ \alpha_t $ berechnet?**


# 1. AdaBoost (Adaptive Boosting)

## Grundidee
AdaBoost passt im Trainingsprozess die **Gewichte der Datenpunkte** an:
- Falsch klassifizierte Beobachtungen werden wichtiger.
- Richtig klassifizierte werden weniger wichtig.

Damit konzentriert sich jeder neue Baum immer mehr auf die schwierigen Beispiele.


# Formale Definition

Wir starten mit Gewichten

$$
w_i^{(1)} = \frac{1}{n}
$$

In jedem Boosting-Schritt trainieren wir einen Klassifikator $ h_t(x) $, typischerweise:

- ein **Decision Stump**, also ein Baum mit Tiefe 1  
- Vorhersagewerte $ h_t(x) \in \{-1, +1\} $


## 1. Gewichteter Fehler

$$
\text{err}_t = \sum_{i=1}^{n} w_i^{(t)} \, \mathbf{1}(h_t(x_i)\neq y_i)
$$


## 2. Modellgewicht (Stärke des Baums)

$$
\alpha_t = \frac{1}{2} \ln\left(\frac{1 - \text{err}_t}{\text{err}_t}\right)
$$

Interpretation:  
- guter Baum $\Rightarrow \text{err}_t$ klein → $\alpha_t$ groß  
- schlechter Baum → $\alpha_t$ klein oder negativ


## 3. Update der Punktgewichte

$$
w_i^{(t+1)} = w_i^{(t)} \, \exp\left( -\alpha_t \, y_i \, h_t(x_i) \right)
$$

Da $ y_i, h_t(x_i) \in \{-1,+1\} $:

- Wenn **richtig klassifiziert** → exponent = −α → Gewicht sinkt  
- Wenn **falsch** → exponent = +α → Gewicht steigt  

Am Ende: Normieren auf Summe 1.


## Finale Vorhersage

$$
F(x) = \mathrm{sign}\left( \sum_{t=1}^T \alpha_t h_t(x) \right)
$$


### Hauptpunkte
1. AdaBoost optimiert direkt die **exponentielle Loss**  
   $$
   L = \sum_i \exp(-y_i F(x_i))
   $$
2. Der Algorithmus “schaut” immer wieder auf dieselben Fehler.
3. Viele schwache Stumps → komplexe, stückweise lineare Entscheidungsgre​nze.


# AdaBoost (Pseudocode)
initialize w = [1/n] * n

for t in range(1, T+1):

    # 1. Train weak learner on weighted data
    h_t = train_stump(X, y, sample_weight=w)
    
    # 2. Weighted error
    err_t = sum(w[i] for i in range(n) if h_t(x_i) != y[i])
    
    # 3. Compute tree strength
    alpha_t = 0.5 * np.log((1 - err_t) / err_t)
    
    # 4. Update sample weights
    for i in range(n):
        w[i] = w[i] * np.exp(-alpha_t * y[i] * h_t(x_i))
    
    # 5. Normalize
    w = w / sum(w)

# Final model:
# F(x) = sign(sum(alpha_t * h_t(x)))


---

# 2. Gradient Boosting – Theorie

Gradient Boosting betrachtet Boosting als **Gradientenabstieg im Funktionsraum**.

Wir versuchen eine Funktion

$$
F(x)
$$

zu finden, die eine Loss-Funktion minimiert:

$$
L = \sum_{i=1}^{n} \ell(y_i, F(x_i))
$$

(z. B. quadratische Loss, Log-Loss, Huber, etc.)


# Kernschritt:
In jedem Boosting-Schritt wird ein Baum auf den **negativen Gradienten** der Loss-Funktion trainiert.


## Beispiel: Regression mit quadratischer Loss
$$
\ell(y, F) = \frac12 (y - F)^2
$$

Gradient der Loss:

$$
\frac{\partial \ell}{\partial F} = F(x_i)-y_i
$$

Negativer Gradient (das, was wir fitten!):

$$
r_i^{(t)} = y_i - F_{t-1}(x_i)
$$


# Algorithmus

**Initialisierung:**

$$
F_0(x) = \arg \min_c \sum_i \ell(y_i, c)
$$

Für quadratische Loss: einfach Mittelwert von y.


**Jeder Boosting-Schritt:**

1. Kompute Residuen  
$$
r_i^{(t)} = - \left[\frac{\partial \ell(y_i, F(x_i))}{\partial F} \right]_{F=F_{t-1}}
$$

2. Fit eines kleinen Regressionsbaums \(h_t(x)\) auf die Residuen

3. Optional pro Blatt eine optimale Schrittweite \(\gamma_{tj}\):  
$$
\gamma_{tj} = \arg\min_\gamma \sum_{x_i \in \text{leaf }j} \ell\left(y_i, F_{t-1}(x_i) + \gamma\right)
$$

4. Update des Modells  
$$
F_t(x) = F_{t-1}(x) + \eta \, h_t(x)
$$


# Finale Interpretation

Gradient Boosting repariert Fehler über **Gradienten der Loss**, nicht über Punktgewichte.

Es ist wesentlich allgemeiner und flexibler als AdaBoost.


# Gradient Boosting (konzeptioneller Pseudocode)

# Step 1: initial model
F = lambda x: np.mean(y)

for t in range(1, T+1):
    
    # 2. Compute negative gradient (residuals)
    residuals = [y[i] - F(x_i) for i in range(n)]
    
    # 3. Fit regression tree to residuals
    h_t = fit_tree(X, residuals, max_depth=3)
    
    # 4. Update model
    F = lambda x, F_prev=F, h=h_t: F_prev(x) + eta * h(x)

# Final model: F(x)


---

# 3. XGBoost – Theorie

XGBoost = Gradient Boosting + Newton-Schritt + Regularisierung + effiziente Baumkonstruktion.


# XGBoost optimiert eine Zielfunktion:

$$
\text{Obj} = 
\underbrace{\sum_{i=1}^{n} \ell(y_i, \hat{y}_i)}_{\text{Data Loss}}
+ 
\underbrace{\sum_{t=1}^{T} \Omega(h_t)}_{\text{Regularisierung}}
$$

mit

$$
\Omega(h_t) = \gamma \cdot B + \frac{\lambda}{2} \sum_{j=1}^{J} w_j^2
$$


## Zweite-Ordnung-Taylor-Approximation

Für jeden Punkt rechnen wir:

$$
g_i = \frac{\partial \ell}{\partial F(x_i)}, 
\qquad
h_i = \frac{\partial^2 \ell}{\partial F(x_i)^2}
$$

Der Baum wird gebaut, indem auf jedem möglichen Split der **Gain** berechnet wird:

$$
\text{Gain} = 
\frac{1}{2}
\left(
\frac{G_L^2}{H_L+\lambda} +
\frac{G_R^2}{H_R+\lambda} -
\frac{(G_L+G_R)^2}{H_L+H_R+\lambda}
\right) - \gamma
$$

mit:

- $G_L = \sum_{i\in L} g_i$
- $H_L = \sum_{i\in L} h_i$

etc.


## Optimaler Blattwert (closed form)

$$
w_j^* = -\frac{G_j}{H_j + \lambda}
$$


## Update-Regel

$$
F_t(x) = F_{t-1}(x) + \eta \cdot h_t(x)
$$


# Was unterscheidet XGBoost?

1. Verwendet **zweite Ableitungen** → Newton-Schritt  
2. Explizite **Regularisierung** pro Baum und Blatt  
3. Sehr effiziente Baum-Split-Berechnung  
4. Ausgefeilte Heuristiken (subsample, colsample, binning)  
5. De facto **State of the Art** bei strukturierten Daten

# High-level pseudocode for XGBoost (simplified for teaching)

initialize F = constant_prediction()

for t in range(1, T+1):
    
    # 1. First and second derivatives of loss
    g = [grad_loss(y[i], F(x_i)) for i in range(n)]
    h = [hess_loss(y[i], F(x_i)) for i in range(n)]
    
    # 2. Build tree maximizing gain using g and h
    tree = build_tree_with_gain(g, h, params)
    
    # 3. Compute optimal leaf weights
    for leaf in tree.leaves:
        G = sum(g[i] for i in leaf.samples)
        H = sum(h[i] for i in leaf.samples)
        leaf.weight = - G / (H + lambda_reg)
    
    # 4. Update model
    F = lambda x, F_prev=F, tr=tree: F_prev(x) + eta * tr(x)

# Final strong model F(x)
